In [13]:
# 安装必要工具
import pandas as pd
from google.colab import files

# 上传文件
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# 读取数据
df = pd.read_csv(file_name)

# 显示前几行确认内容
df.head()


Saving amazon_reviews_final_cleaned_v2.csv to amazon_reviews_final_cleaned_v2.csv


,brand,text,created_at,label,rating,lang,cleaned_text
0,Apple,"No charger. Every thing is good about iPhones,...",2024-11-08,positive,4,en,charger every thing good iphones theres nothin...
1,Apple,"iPhone 13 256GB. It look so fabulous, I am and...",2024-08-16,positive,5,en,iphone gb look fabulous android user switched ...
2,Apple,Flip camera option nill. I tried to flip camer...,2024-05-14,positive,4,en,flip camera option nill tried flip camera reco...
3,Apple,Good product. Happy to get the iPhone 13 in Am...,2024-05-18,positive,5,en,good product happy get iphone amazon offer
4,Apple,Too smooth and effective battery life. 5 star ...,2024-05-14,positive,5,en,smooth effective battery life star


In [14]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# 显示当前数据集中各品牌 + 情绪分布
print("原始标签分布：")
print(df.groupby(['brand', 'label']).size())

# 映射标签为数字编码（与前面保持一致）
label2id = {'negative': 0, 'neutral': 1, 'positive': 2}
df = df[df['label'].isin(label2id)]  # 去掉任何不属于这三类的
df['label_encoded'] = df['label'].map(label2id)

# 分品牌进行划分
train_list = []
val_list = []

for brand in df['brand'].unique():
    brand_df = df[df['brand'] == brand]
    train, val = train_test_split(
        brand_df, test_size=0.2, random_state=42, stratify=brand_df['label_encoded']
    )
    train_list.append(train)
    val_list.append(val)

# 合并训练集与验证集
train_df = pd.concat(train_list).reset_index(drop=True)
val_df = pd.concat(val_list).reset_index(drop=True)

# 显示数量
print(f"训练集数量：{len(train_df)}")
print(f"验证集数量：{len(val_df)}")
print("标签编码映射：", label2id)


原始标签分布：
brand    label   
Apple    negative      534
         neutral       172
         positive     1193
Xiaomi   negative      798
         neutral       314
         positive     1799
samsung  negative     6016
         neutral      1643
         positive    14065
dtype: int64
训练集数量：21226
验证集数量：5308
标签编码映射： {'negative': 0, 'neutral': 1, 'positive': 2}


In [15]:
# 安装 transformers（如果还没装）
!pip install transformers -q

from transformers import BertTokenizer

# 加载 BERT 分词器
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 去除空值或仅包含空格的文本行
train_df = train_df[train_df['cleaned_text'].notnull() & (train_df['cleaned_text'].str.strip() != "")]
val_df = val_df[val_df['cleaned_text'].notnull() & (val_df['cleaned_text'].str.strip() != "")]

# 将文本列转为字符串类型并转为列表
train_texts = train_df['cleaned_text'].astype(str).tolist()
val_texts = val_df['cleaned_text'].astype(str).tolist()

# 提取标签
train_labels = train_df['label_encoded'].tolist()
val_labels = val_df['label_encoded'].tolist()

# 编码文本（加 padding 和截断）
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)


In [16]:
import torch
from torch.utils.data import Dataset

# 自定义 Dataset 类
class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# 创建 Dataset 对象
train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)


In [17]:
import os
os.environ["WANDB_DISABLED"] = "true"

from transformers import BertForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score
import itertools
import pandas as pd

# ⚠️ 请确保 train_dataset, val_dataset, train_labels 已准备好

# 定义评估函数
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")
    return {"accuracy": acc, "f1": f1}

# 参数组合
batch_sizes = [16, 32]
learning_rates = [5e-5, 3e-5]
weight_decays = [0.01]
epochs = [2, 3]

param_combinations = list(itertools.product(batch_sizes, learning_rates, weight_decays, epochs))

results = []

for i, (batch_size, lr, wd, ep) in enumerate(param_combinations):
    print(f"\n🔁 正在运行第 {i+1}/{len(param_combinations)} 个组合")
    print(f"➡️ 参数：batch_size={batch_size}, learning_rate={lr}, weight_decay={wd}, epochs={ep}")

    model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(set(train_labels)))

    training_args = TrainingArguments(
        output_dir=f"./results/run_{i}",
        num_train_epochs=ep,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=32,
        warmup_steps=100,
        weight_decay=wd,
        learning_rate=lr,
        evaluation_strategy="epoch",
        logging_dir=f"./logs/run_{i}",
        logging_steps=50,
        save_strategy="no",
        report_to="none",  # 防止 Colab 报错
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    eval_result = trainer.evaluate()

    print(f"✅ 结果：Accuracy={eval_result['eval_accuracy']:.4f}, F1={eval_result['eval_f1']:.4f}")
    results.append({
        "batch_size": batch_size,
        "learning_rate": lr,
        "weight_decay": wd,
        "epochs": ep,
        "accuracy": eval_result["eval_accuracy"],
        "f1": eval_result["eval_f1"]
    })

# 📊 输出结果表格（按 F1 分数排序）
results_df = pd.DataFrame(results).sort_values(by="f1", ascending=False).reset_index(drop=True)
print("\n📊 所有参数组合结果（按F1分数排序）：")
print(results_df)



🔁 正在运行第 1/8 个组合
➡️ 参数：batch_size=16, learning_rate=5e-05, weight_decay=0.01, epochs=2


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.408700,0.400971,0.849661,0.832291
2,0.332900,0.409496,0.855878,0.846670


✅ 结果：Accuracy=0.8559, F1=0.8467

🔁 正在运行第 2/8 个组合
➡️ 参数：batch_size=16, learning_rate=5e-05, weight_decay=0.01, epochs=3


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


...

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.417700,0.400178,0.848154,0.826677
2,0.333300,0.395050,0.853994,0.842937


✅ 结果：Accuracy=0.8540, F1=0.8429

🔁 正在运行第 8/8 个组合
➡️ 参数：batch_size=32, learning_rate=3e-05, weight_decay=0.01, epochs=3


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.425800,0.406376,0.848907,0.827941
2,0.344600,0.392330,0.849096,0.835939
3,0.252800,0.422016,0.859457,0.852644


✅ 结果：Accuracy=0.8595, F1=0.8526

📊 所有参数组合结果（按F1分数排序）：
   batch_size  learning_rate  weight_decay  epochs  accuracy        f1
0          32        0.00003          0.01       3  0.859457  0.852644
1          16        0.00003          0.01       3  0.852864  0.847636
2          16        0.00005          0.01       2  0.855878  0.846670
3          16        0.00005          0.01       3  0.850791  0.846170
4          16        0.00003          0.01       2  0.857008  0.845934
5          32        0.00005          0.01       3  0.848531  0.844741
6          32        0.00003          0.01       2  0.853994  0.842937
7          32        0.00005          0.01       2  0.853806  0.842090


In [19]:

from collections import Counter

label_distribution = Counter(df['label'])  # 原始数据集中，统计三类标签数量
print("标签分布：", label_distribution)


标签分布： Counter({'positive': 17057, 'negative': 7348, 'neutral': 2129})
